In [1]:
import numpy as np
import pandas as pd
import bloscpack as bp
from sklearn.model_selection import StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import imgaug as ia
import imgaug.augmenters as iaa

In [2]:
import fastai
from fastai.vision import *

from optim import Over9000

In [3]:
from torch.utils.data.dataloader import DataLoader
from data import Bengaliai_DS, Balanced_Sampler

In [4]:
from model import *
from model_utils import *

---

In [5]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

---
### data

#### stratification

In [6]:
pdf = pd.read_csv('../input/train.csv')

In [7]:
pdf.head(3)

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী


In [8]:
unique_grapheme = pdf['grapheme'].unique()
grapheme_code = dict([(g, c) for g, c in zip(unique_grapheme, np.arange(unique_grapheme.shape[0]))])
pdf['grapheme_code'] = [grapheme_code[g] for g in pdf['grapheme']]

In [9]:
# skf = StratifiedKFold(n_splits=5)
# for trn_ndx, vld_ndx in skf.split(pdf['grapheme_code'], pdf['grapheme_code']):
#     break

skf = MultilabelStratifiedKFold(n_splits=4, shuffle=True, random_state=42)
for i, (trn_ndx, vld_ndx) in enumerate(skf.split(pdf['image_id'].values.reshape(-1, 1), pdf.loc[:, ['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values.reshape(-1, 3))):
    if i == 3:
        break

In [10]:
trn_pdf = pdf.iloc[trn_ndx, :]
trn_pdf.reset_index(inplace=True, drop=True)
trn_pdf.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme,grapheme_code
0,Train_2,22,3,5,খ্রী,2
1,Train_3,53,2,2,র্টি,3
2,Train_4,71,9,5,থ্রো,4
3,Train_5,153,9,0,স্পো,5
4,Train_6,52,2,0,ঞ্জি,6


In [8]:
imgs = bp.unpack_ndarray_from_file('../features/train_images.bloscpack')
# see preprocess notebook, it's shown that the train.csv data sequence is the same and can just load like this.
lbls = pd.read_csv('../input/train.csv').iloc[:, 1:4].values

In [12]:
trn_imgs = imgs[trn_ndx]
trn_lbls = lbls[trn_ndx]

vld_imgs = imgs[vld_ndx]
vld_lbls = lbls[vld_ndx]

In [13]:
trn_imgs.shape

(150630, 128, 128)

#### augmentation

In [14]:
augs = iaa.SomeOf(
    (0, 2),
    [
        iaa.SomeOf(
            (1, 2),
            [
                iaa.Affine(scale={"x": (0.8, 1.1), "y": (0.8, 1.1)}, rotate=(-15, 15), shear=(-15, 15)),
                iaa.PiecewiseAffine(scale=(0.02, 0.03)),
            ],
            random_order=True
        ),
        iaa.OneOf(
            [
                iaa.DirectedEdgeDetect(alpha=(.6, .8), direction=(0.0, 1.0)),
                iaa.Emboss(alpha=(.5, 1.), strength=(.1, 4)),
            ]
        ),
    ],
    random_order=True
)

In [15]:
sampler = Balanced_Sampler(trn_pdf, count_column='image_id', primary_group='grapheme_root', secondary_group=['vowel_diacritic', 'consonant_diacritic'], size=150630)

In [16]:
training_set = Bengaliai_DS(trn_imgs, trn_lbls, transform=augs)
validation_set = Bengaliai_DS(vld_imgs, vld_lbls)

training_loader = DataLoader(training_set, batch_size=64, num_workers=6, sampler=sampler)
validation_loader = DataLoader(validation_set, batch_size=64, num_workers=6, shuffle=False)

data_bunch = DataBunch(train_dl=training_loader, valid_dl=validation_loader)

---
### model

In [17]:
device = 'cuda:0'
n_grapheme = 168
n_vowel = 11
n_consonant = 7
n_total = n_grapheme + n_vowel + n_consonant

In [18]:
predictor = PretrainedCNN(out_dim=n_total)
classifier = BengaliClassifier(predictor)#.to(device)

In [19]:
learn = Learner(
    data_bunch,
    classifier,
    loss_func=Loss_combine(),
    opt_func=Over9000,
    metrics=[Metric_grapheme(), Metric_vowel(), Metric_consonant(), Metric_tot()]
)

logger = CSVLogger(learn, 'Seresnext50_Fastai_Selectedaugs_Multilabelstrat_Bsampler_Mixup_fold3of4')

learn.clip_grad = 1.0
learn.split([classifier.predictor.lin_layers])
learn.unfreeze()

In [20]:
learn.fit_one_cycle(
    32,
    max_lr=slice(0.2e-2,1e-2),
    wd=[1e-3, 0.1e-1],
    pct_start=0.0,
    div_factor=100,
    callbacks=[logger, SaveModelCallback(learn, monitor='metric_tot', mode='max', name='Seresnext50_Fastai_Selectedaugs_Multilabelstrat_Bsampler_Mixup_fold3of4'), MixUpCallback(learn)]
)

epoch,train_loss,valid_loss,metric_idx,metric_idx,metric_idx,metric_tot,time
0,2.375793,1.077557,0.867865,0.942654,0.800275,0.869664,08:03
1,2.076872,1.049646,0.888891,0.949942,0.864375,0.898025,07:59
2,1.900371,0.739828,0.911464,0.963105,0.921902,0.926984,08:03
3,1.738501,0.703115,0.926690,0.965045,0.958357,0.944196,08:02
4,1.605984,0.732676,0.931228,0.967369,0.954957,0.946195,08:00
5,1.528176,0.808627,0.936737,0.977431,0.962031,0.953234,08:03
6,1.459820,0.750063,0.938416,0.977018,0.961660,0.953877,07:59
7,1.426961,0.652829,0.940225,0.977310,0.964568,0.955582,08:00
8,1.365748,0.662593,0.942899,0.978949,0.964240,0.957247,08:00
9,1.320455,0.683427,0.946504,0.976849,0.963247,0.958276,08:01


KeyboardInterrupt: 